In [1]:
import pandas as pd
import json
import numpy as np
import re
from nltk.stem.porter import *
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn import model_selection,metrics,naive_bayes,preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
stemmer = PorterStemmer()
import warnings
warnings.filterwarnings('ignore')

In [2]:
def clean_data(X):
    STOPWORDS = set(stopwords.words('english'))
    X=X.str.lower()
    X=X.str.replace("[/(){}\[\]\|@,;]", " ")
    X=X.str.replace("[^0-9a-z #+_]", " ")
    X = X.str.replace(r'\d+','')
    X = X.apply(lambda x: ' '.join([w for w in x.split() if (len(w)>2 and w not in STOPWORDS) ] ))
    X = X.apply(lambda x: x.split()) 
    return X

def target_arrange(y):
    
    for i in range(len(y)):
        if y.values[i]=="Negative":
            y.values[i]=0.0
        elif y[i]=="Positive":
            y.values[i]=1.0
        else:
            y.values[i]=2.0
            
    y=y.to_numpy()  
    y=y.reshape(y.shape[0],1)
    y= pd.DataFrame(data=y)
    y=np.ravel(y)
    y=y.astype('float')
    return y

In [3]:
with open("All_Tickers.json","r") as fp:
#with open("General_Market.json",encoding='utf8') as fp:
    json_d = json.load(fp)
 
ticks_d = json_d['data']
df = pd.DataFrame(ticks_d)

X= pd.DataFrame(columns=['Date', 'Article','Target'])
X['Date']=pd.to_datetime(df['date'])
X['Article']=df['title']+" "+df['text']
X['Target']=df['sentiment']

X=X.sort_values("Date")

print("Number of Examples : ",len(X),"\n")
X.drop_duplicates(inplace=True)
X.index = range(len(X))
print("Number of Examples after removing duplicates: ",len(X),"\n")


X.to_csv (r'General.csv', index = False, header=True)

print('Number of words before cleaning : ',X['Article'].apply(lambda x: len(str(x).split(' '))).sum())
X['Article']=clean_data(X['Article'])
print('Number of words after cleaning : ',X['Article'].apply(lambda x: len(str(x).split(' '))).sum())

print(X.groupby(['Target']).count(),"\n")
print(X['Date'])
y=target_arrange(X['Target'])



Number of Examples :  28130 

Number of Examples after removing duplicates:  27937 

Number of words before cleaning :  1067277
Number of words after cleaning :  718391
           Date  Article
Target                  
Negative   3143     3143
Neutral   14893    14893
Positive   9901     9901 

0       2020-08-07 08:00:00-04:00
1       2020-08-07 08:00:00-04:00
2       2020-08-07 08:00:00-04:00
3       2020-08-07 08:00:00-04:00
4       2020-08-07 08:00:00-04:00
                   ...           
27932   2020-09-04 14:33:55-04:00
27933   2020-09-04 14:34:56-04:00
27934   2020-09-04 14:35:00-04:00
27935   2020-09-04 14:35:12-04:00
27936   2020-09-04 14:39:51-04:00
Name: Date, Length: 27937, dtype: datetime64[ns, pytz.FixedOffset(-240)]


In [4]:
X['Article']= X['Article'].apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
for i in range(len(X['Article'])): #φέρνω τα tokens ξανά μαζί διαχωριζόμενα με κενά
    X['Article'][i] = ' '.join(X['Article'][i])

print(X['Article'])

0        vianet group inc announc unaudit second quarte...
1        krato present canaccord virtual growth confer ...
2        rewalk robot report second quarter financi res...
3        pyxi tanker announc date releas second quarter...
4        bionano genom report second quarter financi re...
                               ...                        
27932    facebook block new polit ad may fall short sti...
27933    amazon growth problem buy good good enough amz...
27934    pyrogenesi sign contract navi two ship build p...
27935    guardant health high price lot potenti guardan...
27936    oil futur post first weekli fall week oil futu...
Name: Article, Length: 27937, dtype: object


In [5]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9)
Xv = tfidf_vectorizer.fit_transform(X['Article'])
Xv = pd.DataFrame(Xv.todense())
X_train,X_test,y_train,y_test = train_test_split(Xv,y, test_size=0.25,stratify=y)

In [6]:
alpha = 0.1 # This is the smoothing parameter for Laplace/Lidstone smoothing
model = naive_bayes.MultinomialNB(alpha=alpha)

model.fit(X_train,y_train)
y_predicted = model.predict(X_test)


recall = metrics.recall_score(y_test,y_predicted,average='macro')
precision = metrics.precision_score(y_test,y_predicted,average='macro')
f1 = metrics.f1_score(y_test,y_predicted,average='macro')
Accur=metrics.accuracy_score(y_test,y_predicted)

print(' Recall metric:',recall)
print(' F1 metric:',f1)
print(' Precision metric:',precision)
print(' Accuracy metric:',Accur)


 Recall metric: 0.6775030111877958
 F1 metric: 0.6862463876004958
 Precision metric: 0.7149336588885277
 Accuracy metric: 0.7570508231925555


In [7]:
skfold = model_selection.StratifiedKFold(n_splits=5)
model2 = RandomForestClassifier(n_estimators=100, max_features="auto", n_jobs=-1)
results = model_selection.cross_val_score(model2, Xv,y, cv=skfold)
print(results.mean())

0.7994405601304491


In [8]:
model2 = RandomForestClassifier(n_estimators=100, max_features="auto", n_jobs=-1)
model2.fit(X_train,y_train)
y_predicted = model2.predict(X_test)


recall = metrics.recall_score(y_test,y_predicted,average='macro')
precision = metrics.precision_score(y_test,y_predicted,average='macro')
f1 = metrics.f1_score(y_test,y_predicted,average='macro')
Accur=metrics.accuracy_score(y_test,y_predicted)

print(' Recall metric:',recall)
print(' F1 metric:',f1)
print(' Precision metric:',precision)
print(' Accuracy metric:',Accur)

 Recall metric: 0.6988502062134616
 F1 metric: 0.7203128953596843
 Precision metric: 0.8137274988487069
 Accuracy metric: 0.8044380816034359
